<a href="https://colab.research.google.com/github/AditMeh/deep-learning/blob/main/meta-learning/siamese_net/train_siamese_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AditMeh/deep-learning

Cloning into 'deep-learning'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 266 (delta 120), reused 90 (delta 34), pack-reused 0
Receiving objects: 100% (266/266), 24.38 MiB | 20.53 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [ ]:
%%capture
%cd ./deep-learning/meta-learning/siamese_net/
!git clone https://github.com/brendenlake/omniglot
%cd omniglot/python
!unzip images_background.zip
!unzip images_evaluation.zip
%cd ../..

In [3]:
!python main.py

Traceback (most recent call last):
  File "main.py", line 14, in <module>
    ds_val = generate_random_pairs(VALIDATION_DATASET_PATH, 10000, train=False)
  File "/content/deep-learning/meta-learning/siamese_net/utils.py", line 10, in generate_random_pairs
    for alphabet in os.listdir(dataset_path):
FileNotFoundError: [Errno 2] No such file or directory: 'omniglot/python/images_evaluation/'
